# 1. Extract Relevant Data 

* From the JSON file, image-name needs to be extarcted and asssocicated classes 
* From the PyTorch similarity results, extract the image names and their highest similarity score along with the associated text prompt 

# 2. Create Mapping 

* Map the image_name from both datasets
* Pair the classes (from teh JSON) with the most rleveant text prompt and ist similarity score (from PyTorch)

# 3. Compare Results 

* For each image_name, check if the classes from the JSON file align with the highest similarity text prompt from PyTorch
* Record matches and mismatches for further analysis


In [1]:
text_prompts = [
    "a photo of a gun", "a photo of a shooting", "a photo of a weapon",
    "a photo of a killing", "a photo of a bomb", "a photo of violence",
    "a photo of a grenade"
    
    # Focus auf waffen 
    #"a photo of other", "a photo of an animal", "a photo of a family",
    #"a photo of a meeting", "a photo of a tool", "a photo of a gathering"
]



In [2]:
# Duplicate and add variations
text_prompts += [
    "a close-up of a gun", "an image of a shooting", "an image of a weapon",
    "an image of a killing", "an image of a bomb", "an image of violence"
]

### Test

In [32]:
import json
import torch

# Step 1: Load JSON and PyTorch results
json_file_path = "weapon_classification_results.json"
torch_file_path = "sim_violence_test.torch"

# Load JSON results
with open(json_file_path, "r") as f:
    weapon_results = json.load(f)

# Load PyTorch similarity results
torch_results = torch.load(torch_file_path, map_location="cpu")

# Step 2: Prepare a mapping from JSON
json_mapping = {
    entry["image_name"]: entry["classes"]
    for entry in weapon_results["results"]
}

# Step 3: Compare with PyTorch results
comparison_results = []

for image_path, similarity_scores in torch_results.items():
    # Convert similarity_scores to a PyTorch tensor if it's not already
    if not isinstance(similarity_scores, torch.Tensor):
        similarity_scores = torch.tensor(similarity_scores)

    # Extract image name from the path
    image_name = image_path.split("/")[-1]

    # Adjust similarity_scores to match the length of text_prompts
    if len(similarity_scores) < len(text_prompts):
        num_dimensions = similarity_scores.dim()
        if num_dimensions == 1:
         # 1D similarity_scores
            similarity_scores = torch.cat(
                (similarity_scores, torch.zeros(len(text_prompts) - len(similarity_scores)))
            )
        elif num_dimensions == 2:
            # 2D similarity_scores
            num_features = similarity_scores.size(1)
            padding = torch.zeros(len(text_prompts) - len(similarity_scores), num_features)
            similarity_scores = torch.cat((similarity_scores, padding), dim=0)
    elif len(similarity_scores) > len(text_prompts):
        similarity_scores = similarity_scores[:len(text_prompts)]

    # Get the highest similarity score and corresponding text prompt
    max_score_idx = similarity_scores.argmax()
    max_score = similarity_scores[max_score_idx].item()  # Convert to scalar here
    related_text = text_prompts[max_score_idx]

    # Check if the image is in the JSON results
    if image_name in json_mapping:
        detected_classes = json_mapping[image_name]
        comparison_results.append({
            "image_name": image_name,
            "detected_classes": detected_classes,
            "torch_related_text": related_text,
            "torch_similarity_score": max_score,
            "match": any(cls.lower() in related_text.lower() for cls in detected_classes)
        })
    else:
        comparison_results.append({
            "image_name": image_name,
            "detected_classes": None,
            "torch_related_text": related_text,
            "torch_similarity_score": max_score,
            "match": False
        })

# Step 4: Output Results
# Step 4: Output Results
for result in comparison_results:
    print(f"Image: {result['image_name']}")
    print(f"  Detected Classes (JSON): {result['detected_classes']}")
    print(f"  Torch Related Text: {result['torch_related_text']} (Scores: {result['torch_similarity_score'].tolist()})")

    print(f"  Torch Related Text: {result['torch_related_text']} (Score: {result['torch_similarity_score'].item():.3f})")
    print(f"  Match: {'Yes' if result['match'] else 'No'}\n")


RuntimeError: a Tensor with 6 elements cannot be converted to Scalar

### Working version

In [12]:
import json
import torch

# Step 1: Load JSON and PyTorch results
json_file_path = "weapon_classification_results.json"
torch_file_path = "sim_violence_test.torch"

# Load JSON results
with open(json_file_path, "r") as f:
    weapon_results = json.load(f)

# Load PyTorch similarity results
torch_results = torch.load(torch_file_path, map_location="cpu")

# Step 2: Prepare a mapping from JSON
json_mapping = {
    entry["image_name"]: entry["classes"]
    for entry in weapon_results["results"]
}

# List of text prompts
text_prompts = [
    "a photo of a gun", "a photo of a shooting", "a photo of a weapon",
    "a photo of a killing", "a photo of a bomb", "a photo of violence",
    "a close-up of a gun", "an image of a shooting", "an image of a weapon",
    "an image of a killing", "an image of a bomb", "an image of violence", "a photo of a knife","a photo of a grenade"
]

# Step 3: Compare with PyTorch results
comparison_results = []

for image_path, similarity_scores in torch_results.items():
    # Convert similarity_scores to a PyTorch tensor if it's not already
    if not isinstance(similarity_scores, torch.Tensor):
        similarity_scores = torch.tensor(similarity_scores)

    # Extract image name from the path
    image_name = image_path.split("/")[-1]

    # Adjust similarity_scores to match the length of text_prompts
    if len(similarity_scores) < len(text_prompts):
        if similarity_scores.dim() == 1:  # 1D similarity_scores
            similarity_scores = torch.cat(
                (similarity_scores, torch.zeros(len(text_prompts) - len(similarity_scores)))
            )
        elif similarity_scores.dim() == 2:  # 2D similarity_scores
            num_features = similarity_scores.size(1)
            padding = torch.zeros(len(text_prompts) - similarity_scores.size(0), num_features)
            similarity_scores = torch.cat((similarity_scores, padding), dim=0)
    elif len(similarity_scores) > len(text_prompts):
        similarity_scores = similarity_scores[:len(text_prompts)]

    # Ensure similarity_scores is 1D for processing
    if similarity_scores.dim() > 1:
        similarity_scores = similarity_scores.flatten()

    # Get the highest similarity score and corresponding text prompt
    max_score_idx = similarity_scores.argmax()  # Index of max score
    max_score = similarity_scores[max_score_idx].item()  # Convert to scalar
    related_text = text_prompts[max_score_idx]

    # Check if the image is in the JSON results
    if image_name in json_mapping:
        detected_classes = json_mapping[image_name]
        comparison_results.append({
            "image_name": image_name,
            "detected_classes": detected_classes,
            "torch_related_text": related_text,
            "torch_similarity_score": max_score,
            "match": any(cls.lower() in related_text.lower() for cls in detected_classes)
        })
    else:
        comparison_results.append({
            "image_name": image_name,
            "detected_classes": None,
            "torch_related_text": related_text,
            "torch_similarity_score": max_score,
            "match": False
        })

# Step 4: Output Results
for result in comparison_results:
    print(f"Image: {result['image_name']}")
    print(f"  Detected Classes (JSON): {result['detected_classes']}")
    print(f"  Torch Related Text: {result['torch_related_text']} (Score: {result['torch_similarity_score']:.3f})")
    print(f"  Match: {'Yes' if result['match'] else 'No'}\n")



Image: id_1080638380466204672_2019-01-03.jpg
  Detected Classes (JSON): ['Knife']
  Torch Related Text: a photo of a killing (Score: 0.171)
  Match: No

Image: id_1080192181284028416_2019-01-01.jpg
  Detected Classes (JSON): ['Knife']
  Torch Related Text: a photo of a killing (Score: 0.140)
  Match: No

Image: id_1080618686275309569_2019-01-03.jpg
  Detected Classes (JSON): ['Gun']
  Torch Related Text: a photo of a shooting (Score: 0.137)
  Match: No

Image: id_1080418713566887936_2019-01-02.jpg
  Detected Classes (JSON): ['Gun']
  Torch Related Text: a photo of a killing (Score: 0.114)
  Match: No

Image: id_1080544662945964033_2019-01-02.jpg
  Detected Classes (JSON): ['Grenade']
  Torch Related Text: a photo of a shooting (Score: 0.137)
  Match: No

Image: id_1080303178569207808_2019-01-02.jpg
  Detected Classes (JSON): ['Baton']
  Torch Related Text: a photo of a weapon (Score: 0.100)
  Match: No

Image: id_1080234390649741312_2019-01-01.jpg
  Detected Classes (JSON): ['Knife']
 

In [13]:
import json

# Step 1: Define the output JSON file path
output_json_path = "../results/comparison_results.json"

# Step 2: Save the comparison results to a JSON file
with open(output_json_path, "w") as json_file:
    json.dump(comparison_results, json_file, indent=4)

print(f"Comparison results saved to {output_json_path}")


Comparison results saved to ../results/comparison_results.json


# Get Top Ten images 

In [14]:
# Extract top 10 images with highest similarity scores
top_images = []

for image_path, similarity_scores in torch_results.items():
    # Get the highest similarity score for this image
    max_score = similarity_scores.max()
    top_images.append((image_path, max_score))

# Sort by similarity score in descending order
top_images = sorted(top_images, key=lambda x: x[1], reverse=True)

# Get the top 10 images
top_10_images = top_images[:10]

# Display the results
print("Top 10 Images with Highest Similarity Scores:")
for rank, (image_path, score) in enumerate(top_10_images, start=1):
    print(f"{rank}. Image: {image_path}, Max Similarity Score: {score:.3f}")



# werte im range 

Top 10 Images with Highest Similarity Scores:
1. Image: /Users/apple/Documents/HIWI_Katharina/Hiwi_Katarina/violence_detection/Katharina_pytorch_Comparison/images_test/id_1081146394398863363_2019-01-04.jpg, Max Similarity Score: 0.197
2. Image: /Users/apple/Documents/HIWI_Katharina/Hiwi_Katarina/violence_detection/Katharina_pytorch_Comparison/images_test/id_1081133255385063425_2019-01-04.jpg, Max Similarity Score: 0.188
3. Image: /Users/apple/Documents/HIWI_Katharina/Hiwi_Katarina/violence_detection/Katharina_pytorch_Comparison/images_test/id_1080970837732347905_2019-01-03.jpg, Max Similarity Score: 0.187
4. Image: /Users/apple/Documents/HIWI_Katharina/Hiwi_Katarina/violence_detection/Katharina_pytorch_Comparison/images_test/id_1080833692732403712_2019-01-03.jpg, Max Similarity Score: 0.184
5. Image: /Users/apple/Documents/HIWI_Katharina/Hiwi_Katarina/violence_detection/Katharina_pytorch_Comparison/images_test/id_1080781129425317888_2019-01-03.jpg, Max Similarity Score: 0.183
6. Image:

### Next Section is still in Progress

In [10]:
# Extract top 10 images with highest similarity scores and their related prompts
top_images_with_prompts = []

for image_path, similarity_scores in torch_results.items():
    max_score_idx = similarity_scores.argmax()
    max_score = similarity_scores[max_score_idx]
    related_text = text_prompts[max_score_idx]
    top_images_with_prompts.append((image_path, max_score, related_text))

# Sort by similarity score in descending order
top_images_with_prompts = sorted(top_images_with_prompts, key=lambda x: x[1], reverse=True)

# Get the top 10 images
top_10_images_with_prompts = top_images_with_prompts[:10]

# Display the results
print("Top 10 Images with Highest Similarity Scores and Related Prompts:")
for rank, (image_path, score, prompt) in enumerate(top_10_images_with_prompts, start=1):
    print(f"{rank}. Image: {image_path}, Max Similarity Score: {score:.3f}, Related Prompt: {prompt}")


IndexError: index 3 is out of bounds for axis 0 with size 1

In [16]:
# Define category indices based on text prompts
category_indices = {
    "gun": text_prompts.index("a photo of a gun"),
    "grenade": text_prompts.index("a photo of a grenade"),  # Add if this was in your prompts
    "knife": text_prompts.index("a photo of a knife")      # Add if this was in your prompts
}

# Function to get top 10 images for a specific category
def get_top_images_by_category(category, index, torch_results):
    category_scores = []

    for image_path, similarity_scores in torch_results.items():
        # Convert similarity_scores to a tensor if not already
        if not isinstance(similarity_scores, torch.Tensor):
            similarity_scores = torch.tensor(similarity_scores)

        # Extract the score for the specified category index
        score = similarity_scores[index].item()  # Convert to scalar value
        category_scores.append((image_path, score))
    
    # Sort by similarity score in descending order
    category_scores = sorted(category_scores, key=lambda x: x[1], reverse=True)
    
    # Get the top 10 images
    return category_scores[:10]

# Get top 10 images for each category
top_10_guns = get_top_images_by_category("gun", category_indices["gun"], torch_results)
top_10_grenades = get_top_images_by_category("grenade", category_indices["grenade"], torch_results)
top_10_knives = get_top_images_by_category("knife", category_indices["knife"], torch_results)

# Display results
print("Top 10 Images for Guns:")
for rank, (image_path, score) in enumerate(top_10_guns, start=1):
    print(f"{rank}. Image: {image_path}, Similarity Score: {score:.3f}")

print("\nTop 10 Images for Grenades:")
for rank, (image_path, score) in enumerate(top_10_grenades, start=1):
    print(f"{rank}. Image: {image_path}, Similarity Score: {score:.3f}")

print("\nTop 10 Images for Knives:")
for rank, (image_path, score) in enumerate(top_10_knives, start=1):
    print(f"{rank}. Image: {image_path}, Similarity Score: {score:.3f}")


RuntimeError: a Tensor with 6 elements cannot be converted to Scalar